Cells Seperated

In [ ]:
from random import random
from numpy import load
from numpy import zeros
from numpy import ones
from numpy import asarray
from numpy.random import randint
from keras.optimizers import Adam
from keras.initializers import RandomNormal
from keras.models import Model
from tensorflow.keras import Input
from keras.layers import Conv2D
from keras.layers import Conv2DTranspose
from keras.layers import LeakyReLU
from keras.layers import Activation
from keras.layers import Concatenate
import matplotlib.pyplot as plt
from os import listdir
#from keras.preprocessing.image import load_img
from numpy import savez_compressed
from keras import backend as K
import os
import numpy as np
from PIL import Image
import pandas as pd
import tensorflow as tf

In [ ]:
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect(tpu="local")
    strategy = tf.distribute.TPUStrategy(tpu)
    print("on TPU")
except tf.errors.NotFoundError:
    print("not on TPU")
    strategy = tf.distribute.MirroredStrategy()

    print("REPLICAS: ", strategy.num_replicas_in_sync)

In [ ]:
with strategy.scope():
    normalized_pixel_values2 = pd.read_pickle('/kaggle/input/normalized1/normalized_pixel_values/npv2_data.pkl')

    normalized_pixel_values1 = pd.read_pickle('/kaggle/input/normalized1/npv1_data.pkl/npv1_data.pkl')
    
    normalized_pixel_values3 = pd.read_pickle('/kaggle/input/normalized1/normalized_pixel_values/npv3_data.pkl')
        
    normalized_pixel_values4 = pd.read_pickle('/kaggle/input/normalized1/normalized_pixel_values/npv4_data.pkl')

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import GroupNormalization
# define layer
layer = GroupNormalization(axis=-1, groups = -1)
print('import successful')


In [ ]:
def discriminator_model(image_shape):
        init = RandomNormal(stddev = 0.02) #weight initialization with a normal distirbution curve
        input_image = Input(shape = image_shape)
        d = Conv2D(64, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(input_image) #64 - number of layers/output channels, (4,4) - size of filter, padding = "same" - ensures output has same dimensions as input after padding
        d = LeakyReLU(alpha=0.2)(d)         

        d = Conv2D(128, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(d)
        d = GroupNormalization(axis=-1, groups = 1)(d)
        d = LeakyReLU(alpha=0.2)(d)

        d = Conv2D(256, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(d)
        d = GroupNormalization(axis=-1, groups = 1)(d)
        d = LeakyReLU(alpha=0.2)(d)

        d = Conv2D(512, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(d)
        d = GroupNormalization(axis=-1, groups = 1)(d)
        d = LeakyReLU(alpha=0.2)(d)

        d = Conv2D(512, (4,4), padding='same', kernel_initializer=init)(d)
        d = GroupNormalization(axis=-1, groups = 1)(d)
        d = LeakyReLU(alpha=0.2)(d)

        patch_out = Conv2D(1, (4,4), padding='same', kernel_initializer=init)(d)
        model = Model(input_image, patch_out)

        model.compile(loss='mse', optimizer=Adam(learning_rate=0.0002, beta_1=0.5), loss_weights=[0.5], metrics=['accuracy']) #lr - learning rate, beta_1 - influences moving average of past gradients, loss_weights - shows relative contribution of mse to the overall loss during training
        print('discriminator model ready')
        return model


In [ ]:
def generator_model(image_shape):
        init = RandomNormal(stddev = 0.02) #weight initialization with a normal distirbution curve
        input_image = Input(shape = image_shape)
        g = Conv2D(64, (7,7), padding = 'same', kernel_initializer = init)(input_image) #64 - number of filters, (7 x 7) - size of filter
        g = GroupNormalization(axis=-1, groups = 1)(g)
        g = Activation('relu')(g)

        g =Conv2D(128, (3,3), strides = (2,2), padding = 'same', kernel_initializer = init)(g)
        g = GroupNormalization(axis=-1, groups = 1)(g)
        g = Activation('relu')(g)

        g =Conv2D(256, (3,3), strides = (2,2), padding = 'same', kernel_initializer = init)(g)
        g = GroupNormalization(axis=-1, groups = 1)(g)
        g = Activation('relu')(g)

        g = Conv2DTranspose(128, (3,3), strides = (2,2), padding = 'same', kernel_initializer = init)(g)
        g = GroupNormalization(axis=-1, groups = 1)(g)
        g = Activation('relu')(g)

        g = Conv2DTranspose(64, (3,3), strides = (2,2), padding = 'same', kernel_initializer = init)(g)
        g = GroupNormalization(axis=-1, groups = 1)(g)
        g = Activation('relu')(g)

        g = Conv2DTranspose(3, (7,7), padding = 'same', kernel_initializer = init)(g)
        g = GroupNormalization(axis=-1, groups = 1)(g)
        output_image = Activation('tanh')(g)
        
        model = Model(input_image, output_image)
        print('generator model ready')
        return model

In [ ]:
def composite_model(g_model_1, d_model, g_model_2, image_shape):
        g_model_1.trainable = True #we need to change the weights of the main generator
        d_model.trainable = False
        g_model_2.trainable = False

        #discriminator element
        input_gen = Input(shape = image_shape)
        gen1_out = g_model_1(input_gen)
        output_d = d_model(gen1_out)

        #identity element
        input_id = Input(shape = image_shape)
        output_id = g_model_1(input_id)

        #forward cycle
        output_f = g_model_2(gen1_out)

        #backward cycle
        gen2_out = g_model_2(input_id)
        output_b = g_model_1(gen2_out)

        model = Model([input_gen, input_id], [output_d, output_id, output_f, output_b])
        optimizer = Adam(learning_rate= 0.0002, beta_1 = 0.5)

        model.compile(loss = ['mse', 'mae', 'mae', 'mae'], loss_weights = [1,5,10,10], optimizer = optimizer, metrics=['accuracy'])
        print('composite model ready')
        return model 

In [ ]:
def generate_real_samples(dataset, n_samples, patch_shape):
         # choose random instances
        ix = randint(0, dataset.shape[0], n_samples)
         # retrieve selected images
        X = dataset[ix]
         # generate 'real' class labels (1)
        y = ones((n_samples, patch_shape, patch_shape, 1))
        print('real sample model ready')
        return X, y


In [ ]:
def generate_fake_samples(g_model, dataset, patch_shape):
     # generate fake instance
        X = g_model.predict(dataset)
     # create 'fake' class labels (0)
        y = zeros((len(X), patch_shape, patch_shape, 1))
        print('fake sample model ready')
        return X, y


In [ ]:
with strategy.scope():
    dataset = [np.concatenate((normalized_pixel_values1, normalized_pixel_values2)), np.concatenate((normalized_pixel_values3, normalized_pixel_values4))]
    dataM, dataP = dataset
    image_shape = (256, 256, 3)
    g_model_MtoP = generator_model(image_shape)
    g_model_PtoM = generator_model(image_shape)
    d_model_M = discriminator_model(image_shape)
    d_model_P = discriminator_model(image_shape)
    c_model_MtoP = composite_model(g_model_MtoP, d_model_P, g_model_PtoM, image_shape)
    c_model_PtoM = composite_model(g_model_PtoM, d_model_M, g_model_MtoP, image_shape)
print('ready')

In [ ]:
from keras.callbacks import ModelCheckpoint
from keras.models import load_model
with strategy.scope():
        # Define hyperparameters
    num_epochs = 2  # Adjust as needed
    batch_size = 1     # You can adjust the batch size
    patch_shape = d_model_M.output_shape[1]   # Adjust the patch size as needed

        # Define a directory to save model checkpoints
    checkpoint_dir = '/kaggle/working/model_checkpoints'  # Adjust the directory as needed

        # Create the directory if it doesn't exist
    import os
    os.makedirs(checkpoint_dir, exist_ok=True)

        # Define a file naming pattern for saved models
    checkpoint_filepath = os.path.join(checkpoint_dir, 'model_epoch_{epoch:03d}.h5')

        # Create a ModelCheckpoint callback to save models
    model_checkpoint_callback = ModelCheckpoint(
        filepath=checkpoint_filepath,
        save_weights_only=True,  # Save only the weights, not the entire model
        monitor='val_loss',     # Monitor a specific metric (e.g., validation loss)
        save_best_only=False,   # Save all models or just the best one
        verbose=1, # Provide progress updates

        )

        # Training loop
        # Training loop
    training_history = {'D_loss_realM': [], 'D_loss_realP': [], 'D_loss_fakeM': [], 'D_loss_fakeP': [], 'G_lossM': [], 'G_lossP': []}  # Initialize an empty dictionary to store the training history

    bat_per_epo = int(len(dataM) / batch_size)
     # calculate the number of training iterations
    n_steps = bat_per_epo * num_epochs
     # manually enumerate epochs
    for epoch in range(n_steps):
                # Train Discriminators
        real_imagesM, real_labelsM = generate_real_samples(dataM, batch_size, patch_shape) #This line generates a batch of real images from the current batch in the dataset and assigns them real labels (usually 1, indicating real images).
        real_imagesP, real_labelsP = generate_real_samples(dataP, batch_size, patch_shape)

        fake_imagesM, fake_labelsM = generate_fake_samples(g_model_PtoM, real_imagesP, patch_shape) #: This line generates a batch of fake images by passing a batch from the dataset through the generator g_model_MtoP and assigns them fake labels (usually 0, indicating fake images).
        fake_imagesP, fake_labelsP = generate_fake_samples(g_model_MtoP, real_imagesM, patch_shape)

        g_lossP = c_model_PtoM.train_on_batch([real_imagesP, real_imagesM], [real_labelsM, real_imagesM, real_imagesP, real_imagesM])

        d_loss_realM = d_model_M.train_on_batch(real_imagesM, real_labelsM)
        d_loss_fakeM = d_model_M.train_on_batch(fake_imagesM, fake_labelsM)

        g_lossM = c_model_MtoP.train_on_batch([real_imagesM, real_imagesP], [real_labelsP, real_imagesP, real_imagesM, real_imagesP])

        d_loss_realP = d_model_P.train_on_batch(real_imagesP, real_labelsP)
        d_loss_fakeP = d_model_P.train_on_batch(fake_imagesP, fake_labelsP)

        # Print training progress
        print(f"Epoch {epoch+1}/{n_steps}, D Loss RealM: {d_loss_realM}, D Loss RealP: {d_loss_realP}, D Loss FakeM: {d_loss_fakeM}, D Loss FakeP: {d_loss_fakeP}, G LossM: {g_lossM}, G LossP: {g_lossP}")
        training_history['D_loss_realM'].append(d_loss_realM)
        training_history['D_loss_realP'].append(d_loss_realP)
        training_history['D_loss_fakeM'].append(d_loss_fakeM)
        training_history['D_loss_fakeP'].append(d_loss_fakeP)
        training_history['G_lossM'].append(g_lossM)
        training_history['G_lossM'].append(g_lossP)



    g_model_MtoP.save('/kaggle/working/generator_model_MtoP.h5')
    # Save the generator model (you can do the same for the discriminator)
    g_model_PtoM.save('/kaggle/working/generator_model_PtoM.h5')
    # Save the generator model (you can do the same for the discriminator)
    d_model_M.save('/kaggle/working/discriminator_model_M.h5')
    # Save the generator model (you can do the same for the discriminator)
    d_model_P.save('/kaggle/working/discriminator_model_P.h5')

    # Save training history (optional)
    import pickle
    with open('/kaggle/working/training_history.pkl', 'wb') as history_file:
        pickle.dump(training_history, history_file)

In [ ]:
import matplotlib.pyplot as plt

input_image = normalized_pixel_values3[:3]
fake_images = g_model_PtoM.predict(input_image)

# Display the first few generated images
num_images_to_display = 3

for i in range(num_images_to_display):
    plt.figure(figsize=(8, 8))
    # Ensure that pixel values are within the range [-1, 1] for displaying
    clipped_image = np.clip(fake_images[i], -1, 1)
    plt.imshow((clipped_image + 1) / 2)  # Normalize to [0, 1] for display
    plt.axis('off')
    plt.show()


In [ ]:
from tensorflow import keras

# Load the entire model
loaded_model = keras.models.load_model('/kaggle/working/discriminator_model_M.h5')

# Print model summary
loaded_model.summary()


In [ ]:
loaded_model = keras.models.load_model('/kaggle/working/generator_model_PtoM.h5')

# Print model summary
loaded_model.summary()


In [ ]:
from tensorflow import keras

# Load the entire model
loaded_model = keras.models.load_model('/kaggle/working/generator_model_PtoM.h5')

# Compile the model manually with the desired optimizer, loss function, and metrics
loaded_model.compile(
    optimizer='adam',  # Replace with your desired optimizer
    loss='mse',        # Replace with your desired loss function
    metrics=['accuracy']  # Replace with your desired metrics
)

# Now the model is compiled and ready for use in training or evaluation


In [ ]:
import pickle

# Load the pickled data from the file
with open('/kaggle/working/training_history.pkl', 'rb') as file:
    training_history = pickle.load(file)

# Now you can work with the loaded data
# For example, you can print the contents of the loaded dictionary
print(training_history)


In [ ]:
import h5py

# Open the H5 file
with h5py.File('/kaggle/working/discriminator_model_M.h5', 'r') as file:
    # List all the top-level groups/datasets in the file
    print("Top-level items in the H5 file:")
    for item in file.keys():
        print(item)

    # Access and print the contents of a specific dataset
    dataset = file['model_weights']
    print("Contents of the dataset:")
    print(dataset[...])  # Use [()] to access the data


In [ ]:

try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print('Device:', tpu.master())
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
except:
    strategy = tf.distribute.get_strategy()
print('Number of replicas:', strategy.num_replicas_in_sync)

AUTOTUNE = tf.data.experimental.AUTOTUNE
    
print(tf.__version__)

In [ ]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Enable GPU memory growth to allocate only the GPU memory needed.
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        print(e)


In [ ]:
strategy = tf.distribute.MirroredStrategy()

In [ ]:
with strategy.scope():
    normalized_pixel_values2 = pd.read_pickle('/kaggle/input/normalized1/normalized_pixel_values/npv2_data.pkl')

    normalized_pixel_values1 = pd.read_pickle('/kaggle/input/normalized1/npv1_data.pkl/npv1_data.pkl')

In [ ]:
with strategy.scope():
    normalized_pixel_values3 = pd.read_pickle('/kaggle/input/normalized1/normalized_pixel_values/npv3_data.pkl')

In [ ]:
with strategy.scope():
    normalized_pixel_values4 = pd.read_pickle('/kaggle/input/normalized1/normalized_pixel_values/npv4_data.pkl')

In [ ]:
print(normalized_pixel_values4)

In [ ]:
print(normalized_pixel_values1[1])

In [ ]:
def data_generator(data, batch_size):
    num_samples = len(data)
    indices = list(range(num_samples))
    while True:
        np.random.shuffle(indices)
        for i in range(0, num_samples, batch_size):
            batch_indices = indices[i:i+batch_size]
            batch_data = data.iloc[batch_indices]
            yield batch_data
print('ready')

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import GroupNormalization
# define layer
layer = GroupNormalization(axis=-1, groups = -1)
print('import successful')

In [ ]:
# Specify which GPU to use (e.g., GPU 0)
with strategy.scope():
    # Define your model here
    # ...

    def discriminator_model(image_shape):
        init = RandomNormal(stddev = 0.02) #weight initialization with a normal distirbution curve
        input_image = Input(shape = image_shape)
        d = Conv2D(64, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(input_image) #64 - number of layers/output channels, (4,4) - size of filter, padding = "same" - ensures output has same dimensions as input after padding
        d = LeakyReLU(alpha=0.2)(d)         

        d = Conv2D(128, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(d)
        d = GroupNormalization(axis=-1, groups = 1)(d)
        d = LeakyReLU(alpha=0.2)(d)

        d = Conv2D(256, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(d)
        d = GroupNormalization(axis=-1, groups = 1)(d)
        d = LeakyReLU(alpha=0.2)(d)

        d = Conv2D(512, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(d)
        d = GroupNormalization(axis=-1, groups = 1)(d)
        d = LeakyReLU(alpha=0.2)(d)

        d = Conv2D(512, (4,4), padding='same', kernel_initializer=init)(d)
        d = GroupNormalization(axis=-1, groups = 1)(d)
        d = LeakyReLU(alpha=0.2)(d)

        patch_out = Conv2D(1, (4,4), padding='same', kernel_initializer=init)(d)
        model = Model(input_image, patch_out)

        model.compile(loss='mse', optimizer=Adam(learning_rate=0.0002, beta_1=0.5), loss_weights=[0.5]) #lr - learning rate, beta_1 - influences moving average of past gradients, loss_weights - shows relative contribution of mse to the overall loss during training

        return model
print('discriminator model ready')

In [ ]:
# Specify which GPU to use (e.g., GPU 0)
with strategy.scope():
    # Define your model here
    # ...

    def generator_model(image_shape):
        init = RandomNormal(stddev = 0.02) #weight initialization with a normal distirbution curve
        input_image = Input(shape = image_shape)
        g = Conv2D(64, (7,7), padding = 'same', kernel_initializer = init)(input_image)
        g = GroupNormalization(axis=-1, groups = 1)(g)
        g = Activation('relu')(g)

        g =Conv2D(128, (3,3), strides = (2,2), padding = 'same', kernel_initializer = init)(g)
        g = GroupNormalization(axis=-1, groups = 1)(g)
        g = Activation('relu')(g)

        g =Conv2D(256, (3,3), strides = (2,2), padding = 'same', kernel_initializer = init)(g)
        g = GroupNormalization(axis=-1, groups = 1)(g)
        g = Activation('relu')(g)

        g = Conv2DTranspose(128, (3,3), strides = (2,2), padding = 'same', kernel_initializer = init)(g)
        g = GroupNormalization(axis=-1, groups = 1)(g)
        g = Activation('relu')(g)

        g = Conv2DTranspose(64, (3,3), strides = (2,2), padding = 'same', kernel_initializer = init)(g)
        g = GroupNormalization(axis=-1, groups = 1)(g)
        g = Activation('relu')(g)

        g = Conv2DTranspose(3, (7,7), padding = 'same', kernel_initializer = init)(g)
        g = GroupNormalization(axis=-1, groups = 1)(g)
        output_image = Activation('tanh')(g)

        model = Model(input_image, output_image)
        return model
print('generator model ready')

In [ ]:
# Specify which GPU to use (e.g., GPU 0)
with strategy.scope():
    # Define your model here
    # ...

    def composite_model(g_model_1, d_model, g_model_2, image_shape):
        g_model_1.trainable = True #we need to change the weights of the main generator
        d_model.trainable = False
        g_model_2.trainable = False

        #discriminator element
        input_gen = Input(shape = image_shape)
        gen1_out = g_model_1(input_gen)
        output_d = d_model(gen1_out)

        #identity element
        input_id = Input(shape = image_shape)
        output_id = g_model_1(input_id)

        #forward cycle
        output_f = g_model_2(gen1_out)

        #backward cycle
        gen2_out = g_model_2(input_id)
        output_b = g_model_1(gen2_out)

        model = Model([input_gen, input_id], [output_d, output_id, output_f, output_b])
        optimizer = Adam(learning_rate= 0.0002, beta_1 = 0.5)

        model.compile(loss = ['mse', 'mae', 'mae', 'mae'], loss_weights = [1,5,10,10], optimizer = optimizer)
        return model 
print('composite model ready')

In [ ]:
with strategy.scope():
# select a batch of random samples, returns images and target
    def generate_real_samples(dataset, n_samples, patch_shape):
     # choose random instances
     ix = randint(0, dataset.shape[0], n_samples)
     # retrieve selected images
     X = dataset[ix]
     # generate 'real' class labels (1)
     y = ones((n_samples, patch_shape, patch_shape, 1))
     return X, y
    print('real sample model ready')

In [ ]:
with strategy.scope():
    # generate a batch of images, returns images and targets
    def generate_fake_samples(g_model, dataset, patch_shape):
     # generate fake instance
     X = g_model.predict(dataset)
     # create 'fake' class labels (0)
     y = zeros((len(X), patch_shape, patch_shape, 1))
     return X, y
print('fake sample model ready')

In [ ]:
with strategy.scope():
    dataset = [np.concatenate((normalized_pixel_values1, normalized_pixel_values2)), np.concatenate((normalized_pixel_values3, normalized_pixel_values4))]
    image_shape = (256, 256, 3)
    g_model_MtoP = generator_model(image_shape)
    g_model_PtoM = generator_model(image_shape)
    d_model_M = discriminator_model(image_shape)
    d_model_P = discriminator_model(image_shape)
    c_model_MtoP = composite_model(g_model_MtoP, d_model_P, g_model_PtoM, image_shape)
    c_model_PtoM = composite_model(g_model_PtoM, d_model_M, g_model_MtoP, image_shape)
print('ready')

In [ ]:
import numpy as np
from keras.callbacks import ModelCheckpoint
with strategy.scope():
    # Define hyperparameters
    num_epochs = 1000  # Adjust as needed
    batch_size = 1     # You can adjust the batch size
    patch_shape = 16   # Adjust the patch size as needed

    # Define a directory to save model checkpoints
    checkpoint_dir = '/kaggle/working/model_checkpoints'  # Adjust the directory as needed

    # Create the directory if it doesn't exist
    import os
    os.makedirs(checkpoint_dir, exist_ok=True)

    # Define a file naming pattern for saved models
    checkpoint_filepath = os.path.join(checkpoint_dir, 'model_epoch_{epoch:03d}.h5')

    # Create a ModelCheckpoint callback to save models
    model_checkpoint_callback = ModelCheckpoint(
        filepath=checkpoint_filepath,
        save_weights_only=True,  # Save only the weights, not the entire model
        monitor='val_loss',     # Monitor a specific metric (e.g., validation loss)
        save_best_only=False,   # Save all models or just the best one
        verbose=1               # Provide progress updates
    )

    # Training loop
    # Training loop
    training_history = {'D_loss_real': [], 'D_loss_fake': [], 'G_loss': []}  # Initialize an empty dictionary to store the training history

    for epoch in range(num_epochs):
        # Iterate over batches of data (assuming you have a dataset object)
        for batch in dataset:  # Replace 'your_dataset' with your actual dataset
            # Train Discriminators
            real_images, real_labels = generate_real_samples(batch, batch_size, patch_shape)
            fake_images, fake_labels = generate_fake_samples(g_model_MtoP, batch, patch_shape)
            d_loss_real = d_model_P.train_on_batch(real_images, real_labels)
            d_loss_fake = d_model_P.train_on_batch(fake_images, fake_labels)

            # Train Generators
            real_images, real_labels = generate_real_samples(batch, batch_size, patch_shape)
            g_loss = c_model_MtoP.train_on_batch([real_images, real_images], [real_labels, real_images, real_images, real_images])

            # Print training progress
            print(f"Epoch {epoch+1}/{num_epochs}, D Loss Real: {d_loss_real}, D Loss Fake: {d_loss_fake}, G Loss: {g_loss}")

        # Save the losses in the training history
        training_history['D_loss_real'].append(d_loss_real)
        training_history['D_loss_fake'].append(d_loss_fake)
        training_history['G_loss'].append(g_loss)

        # Save model checkpoints after each epoch
        model_checkpoint_callback.on_epoch_end(epoch, logs={'val_loss': g_loss})  # Save the model



# Finalize training
# ... (training loop and ModelCheckpoint as mentioned previously)

# Finalize training
# Save the generator and discriminator models
g_model_MtoP.save('/kaggle/working/generator_model_MtoP.h5')
# Save the generator model (you can do the same for the discriminator)
g_model_PtoM.save('/kaggle/working/generator_model_PtoM.h5')
# Save the generator model (you can do the same for the discriminator)
d_model_M.save('/kaggle/working/discriminator_model_M.h5')
# Save the generator model (you can do the same for the discriminator)
d_model_P.save('/kaggle/working/discriminator_model_P.h5')

# Save training history (optional)
import pickle
with open('/kaggle/working/training_history.pkl', 'wb') as history_file:
    pickle.dump(training_history, history_file)

# Optionally, you can perform additional tasks, such as evaluation or inference

